In [1]:
import numpy as np
import cma
import csv
import os
from itertools import product
from functools import partial
from scipy import stats

In [2]:
def parse_csv(filepath):
    with open(filepath) as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        line_count = 1
        episodes = []
        m = int(next(csv_reader)[0])
        num_actions = int(next(csv_reader)[0])
        k = int(next(csv_reader)[0])
        state_rep_size = (k+1)**m
        theta_b = np.array(next(csv_reader), dtype=np.float).reshape(num_actions, state_rep_size).T
        n = int(next(csv_reader)[0])
        for _ in range(n):
            episodes.append(np.array(next(csv_reader), dtype=np.float))
        policy_test_expected = np.array(next(csv_reader), dtype=np.float)
        return m, num_actions, k, theta_b, episodes, policy_test_expected    

def softmax(x):
    return np.exp(x)/sum(np.exp(x))

def pi(theta,c,s):
    phi_s = np.cos(np.pi * np.dot(c,s)).flatten()
    return softmax(np.dot(phi_s,theta))

def PDIS(H, pi_e, pi_b, gamma=1):
    L = int(len(H)/3)
    S = H[0::3]
    A = H[1::3].astype(int)
    R = H[2::3]
    pi_e_array = np.array([pi_e(S[i])[A[i]] for i in range(L)])
    pi_b_array = np.array([pi_b(S[i])[A[i]] for i in range(L)])
    importance_weights = pi_e_array / pi_b_array
    prod_importance_weights = np.cumprod(importance_weights)
    gamma_array = gamma**np.arange(L)
    return np.sum(gamma_array * prod_importance_weights * R)

def B(PDIS_array, delta=0.01):
    n = len(PDIS_array)
    return np.mean(PDIS_array) - (np.std(PDIS_array, ddof=1) / np.sqrt(n) * stats.t.ppf(1-delta, n-1))

## Test policy representation

In [3]:
m, num_actions, k, theta_b, episodes, policy_test_expected = parse_csv('data.csv')
c = np.flip(list(product(range(k+1), repeat=m)), axis=1)
pi_b = partial(pi,theta_b,c)

H_test = episodes[0]
S_test = H_test[0::3]
A_test = H_test[1::3].astype(int)
policy_test_actual = np.array([pi_b(S_test[i])[A_test[i]] for i in range(len(S_test))])

print(f'num state features (m): {m}')
print(f'num actions: {num_actions}')
print(f'fourier basis order (k): {k}')
print(f'theta_b: \n{theta_b}')
print(f'num episodes: {len(episodes)}')
print(f'first episode: {episodes[0]}')
print(f'policy expected: {policy_test_expected}')
print(f'policy actual: {policy_test_actual}')

num state features (m): 1
num actions: 2
fourier basis order (k): 1
theta_b: 
[[ 0.01  1.  ]
 [-0.01  1.  ]]
num episodes: 200000
first episode: [ 0.419908   1.         0.992628   0.366283   0.        10.
  0.0622811  1.         2.98566    0.327772   1.         1.69524
  0.612293   0.         6.10134  ]
policy expected: [0.775818 0.197512 0.878759 0.819091 0.345012]
policy actual: [0.77581796 0.19751227 0.87875879 0.81909146 0.34501192]


## Run HCOPE

In [4]:
Dc = episodes[:100000]
Ds = episodes[100000:]
safety_c = np.mean([np.sum(episode[2::3]) for episode in episodes])
num_solutions = 25

def objective_function(theta_c):
    pi_c = partial(pi,theta_c.reshape([2,2]),c)
    PDIS_array = np.array([PDIS(episode, pi_c, pi_b) for episode in Dc])
    return -np.mean(PDIS_array)

def passes_safety_test(theta_c):
    pi_c = partial(pi,theta_c.reshape([2,2]),c)
    PDIS_array = np.array([PDIS(episode, pi_c, pi_b) for episode in Ds])
    return B(PDIS_array) > safety_c

submission_folder = 'submission'
if not os.path.exists(submission_folder):
    os.mkdir(submission_folder)

for i in range(num_solutions):
    candidate_solution = cma.fmin(objective_function, theta_b.flatten(), 1, options={'maxiter': 5})[0]
    if passes_safety_test(candidate_solution): 
        print(f'passed: {candidate_solution}')
        np.savetxt(f'{submission_folder}/{i+1}.csv', candidate_solution.reshape([1,-1]), delimiter=",")
        np.savetxt(f'{submission_folder}/{i+26}.csv', candidate_solution.reshape([1,-1]), delimiter=",")
        np.savetxt(f'{submission_folder}/{i+51}.csv', candidate_solution.reshape([1,-1]), delimiter=",")
        np.savetxt(f'{submission_folder}/{i+76}.csv', candidate_solution.reshape([1,-1]), delimiter=",")
    else: 
        print(f'failed: {candidate_solution}')

(4_w,8)-aCMA-ES (mu_w=2.6,w_1=52%) in dimension 4 (seed=380130, Fri Dec 13 15:38:44 2019)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1      8 -1.150264677012713e+01 1.0e+00 1.20e+00  1e+00  1e+00 1:48.7
    2     16 -1.467070940085246e+01 1.3e+00 1.60e+00  1e+00  2e+00 3:35.8
    3     24 -1.482766724408732e+01 1.4e+00 2.13e+00  2e+00  2e+00 5:22.0
    4     32 -1.491159737250455e+01 1.5e+00 2.62e+00  2e+00  3e+00 7:11.2
    5     40 -1.491332029610201e+01 1.6e+00 3.26e+00  3e+00  4e+00 8:58.1
termination on maxiter=5 (Fri Dec 13 15:47:57 2019)
final/bestever f-value = -1.491327e+01 -1.491332e+01
incumbent solution: [7.725373007197602, -6.398119725412064, -1.5519083918766061, 2.271136415128719]
std deviation: [3.7791312847704295, 3.3310110023920383, 3.0119048458392723, 2.637435295224247]
passed: [ 8.10780374 -5.91309241 -1.20772353  1.81661436]
(4_w,8)-aCMA-ES (mu_w=2.6,w_1=52%) in dimension 4 (seed=402740, Fri Dec 13 15:48:12 2019)
Iterat #Fevals   fun

    2     16 -1.175006464667542e+01 1.4e+00 1.11e+00  1e+00  1e+00 3:41.5
    3     24 -1.428377185452987e+01 1.5e+00 1.22e+00  1e+00  1e+00 5:27.8
    4     32 -1.490498607189337e+01 1.8e+00 1.75e+00  1e+00  2e+00 10:32.5
    5     40 -1.491326178746005e+01 1.8e+00 2.57e+00  2e+00  3e+00 12:22.9
termination on maxiter=5 (Fri Dec 13 17:18:52 2019)
final/bestever f-value = -1.491322e+01 -1.491326e+01
incumbent solution: [4.741334476740636, -7.119586057524452, 0.7118611884894954, -2.7822040021174548]
std deviation: [2.691610441074851, 3.125060447119746, 2.1987196015466317, 2.464230963700791]
passed: [ 4.99830126 -7.12315927 -0.53732036 -4.06944348]
(4_w,8)-aCMA-ES (mu_w=2.6,w_1=52%) in dimension 4 (seed=447919, Fri Dec 13 17:19:06 2019)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1      8 -4.962394491139076e+00 1.0e+00 8.98e-01  8e-01  9e-01 1:53.2
    2     16 -1.244956883587312e+01 1.1e+00 1.11e+00  1e+00  1e+00 3:52.5
    3     24 -1.485496307326472e+01

    5     40 -1.491096715487132e+01 2.1e+00 2.22e+00  2e+00  3e+00 9:48.4
termination on maxiter=5 (Fri Dec 13 18:55:39 2019)
final/bestever f-value = -1.489783e+01 -1.491097e+01
incumbent solution: [3.8521937869591927, -3.526019395520713, 4.455721791765213, 2.467486806299881]
std deviation: [2.0652569348930174, 2.7946116000123897, 2.62101791507892, 2.0411791868185962]
passed: [ 4.99422808 -4.26516494  4.68691447  1.59164409]
(4_w,8)-aCMA-ES (mu_w=2.6,w_1=52%) in dimension 4 (seed=319363, Fri Dec 13 18:55:54 2019)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1      8 -1.332903074240849e+01 1.0e+00 1.17e+00  1e+00  1e+00 1:56.7
    2     16 -1.480950824535203e+01 1.4e+00 1.72e+00  2e+00  2e+00 3:54.8
    3     24 -1.491058819668025e+01 1.8e+00 2.27e+00  2e+00  3e+00 5:52.6
    4     32 -1.491340510102422e+01 1.9e+00 3.28e+00  3e+00  5e+00 7:51.7
    5     40 -1.491340558735549e+01 2.1e+00 4.95e+00  4e+00  7e+00 9:59.5
termination on maxiter=5 (Fri Dec 13 1